# DI725 Final Assignment Notebook
This notebook orchestrates training for:
- `nanoGPT` from scratch
- `GPT-2` fine-tuning

It uses modular `.py` scripts and logs metrics to WANDB.

## 1. Install Required Packages

In [1]:
#pip install torch transformers datasets evaluate wandb pandas scikit-learn seaborn

## 2. Login to Weights & Biases

In [2]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: esrasekerci (esrasekerci-metu-middle-east-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 3. Train `nanoGPT` from Scratch

In [3]:
!python3 train_sentiment_nanogpt.py

wandb: Currently logged in as: esrasekerci (esrasekerci-metu-middle-east-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /Users/esrasekerci/Desktop/DI725/assignments/assignment_1/wandb/run-20250406_225321-2yfgtvbg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run defiant-spot-19
wandb: ⭐️ View project at https://wandb.ai/esrasekerci-metu-middle-east-technical-university/di725-sentiment-transformer-restore
wandb: 🚀 View run at https://wandb.ai/esrasekerci-metu-middle-east-technical-university/di725-sentiment-transformer-restore/runs/2yfgtvbg
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Nested

## 4. Fine-Tune Pretrained GPT-2

In [4]:
!python3 train_gpt2.py

wandb: Currently logged in as: esrasekerci (esrasekerci-metu-middle-east-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /Users/esrasekerci/Desktop/DI725/assignments/assignment_1/wandb/run-20250406_225537-2xuynldk
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run gpt2-final
wandb: ⭐️ View project at https://wandb.ai/esrasekerci-metu-middle-east-technical-university/di725-sentiment-gpt2
wandb: 🚀 View run at https://wandb.ai/esrasekerci-metu-middle-east-technical-university/di725-sentiment-gpt2/runs/2xuynldk
Map: 100%|███████████████████████████| 271/271 [00:00<00:00, 1152.45 examples/s]
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library

In [5]:
import pandas as pd
import re
import string
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.metrics import classification_report

from model import SentimentTransformer

# === Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Label mapping
sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
inv_map = {v: k for k, v in sentiment_map.items()}

# === Load test data
df_test = pd.read_csv("data/test.csv")
df_test.columns = df_test.columns.str.lower().str.strip()

# === Preprocessing (same as training)
custom_phrases = [
    r'\bis there anything else i can (assist|help) you with\b',
    r'\bthank you for choosing brownbox\b',
    r'\bthank you\b',
    r'\byoure welcome\b',
    r'\btake care\b',
    r'\bgoodbye\b',
    r'\bplease\b',
    r'\bthanks\b',
    r'\bsure\b',
    r'\bno thats all\b',
    r'\bhave a (nice|great|good) day\b',
    r'\bappreciate\b',
    r'\bfor contacting brownbox customer support\b'
]

misspellings = {
    'ts': 'this', 'witn': 'within', 'anytng': 'anything',
    'ithis': 'it has', 'thathis': 'that is', 'as you': 'assure you',
    'en that': 'ensure that'
}

def clean_conversation(text):
    text = text.lower()
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\+?\d[\d\s\-().]{8,}\d', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)

    lines = text.strip().split('\n')
    if lines and lines[0].startswith("agent:"):
        if any(greet in lines[0] for greet in [
            "thank you for calling", "hi", "hello", "this is", "my name is",
            "how can i help you", "how may i assist you"
        ]):
            lines = lines[1:]

    text = ' '.join(lines)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()

    for phrase in custom_phrases:
        text = re.sub(phrase, '', text)

    for wrong, right in misspellings.items():
        text = text.replace(wrong, right)

    text = re.sub(r'\b(customer|agent)\b\s*$', '', text)
    return text

df_test["text"] = df_test["conversation"].astype(str).apply(clean_conversation)
df_test["label"] = df_test["customer_sentiment"].str.strip().str.lower().map(sentiment_map)

# === Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
BLOCK_SIZE = 128

# === Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.encodings = tokenizer(
            list(texts),
            truncation=True,
            padding="max_length",
            max_length=max_len,
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx],
        }

# === DataLoader
test_dataset = TextDataset(df_test["text"], df_test["label"], tokenizer, BLOCK_SIZE)
test_loader = DataLoader(test_dataset, batch_size=32)

# === NanoGPT Evaluation
model_nano = SentimentTransformer(
    vocab_size=tokenizer.vocab_size,
    emb_dim=256,
    max_len=BLOCK_SIZE
).to(device)

model_nano.load_state_dict(torch.load("sentiment_transformer_best.pt", map_location=device))
model_nano.eval()

all_preds_nano, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attn_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model_nano(input_ids, attn_mask)
        preds = torch.argmax(logits, dim=1)

        all_preds_nano.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

print("📊 NanoGPT Evaluation:")
print(classification_report(all_labels, all_preds_nano,
                            target_names=[inv_map[i] for i in sorted(inv_map)], digits=4))

# === GPT-2 Evaluation
model_gpt2 = GPT2ForSequenceClassification.from_pretrained("gpt2_sentiment_model").to(device)
model_gpt2.eval()

all_preds_gpt2 = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attn_mask = batch["attention_mask"].to(device)

        outputs = model_gpt2(input_ids=input_ids, attention_mask=attn_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds_gpt2.extend(preds.cpu().tolist())

print("📊 GPT-2 Evaluation:")
print(classification_report(all_labels, all_preds_gpt2,
                            target_names=[inv_map[i] for i in sorted(inv_map)], digits=4))

/var/folders/zn/vf7dcg9j5yx547py9_y411jw0000gn/T/ipykernel_69377/986360170.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_nano.load_state_dict(torch.load("senti

📊 NanoGPT Evaluation:
              precision    recall  f1-score   support

    negative     0.8889    0.8000    0.8421        10
     neutral     0.5000    0.9000    0.6429        10
    positive     0.6667    0.2000    0.3077        10

    accuracy                         0.6333        30
   macro avg     0.6852    0.6333    0.5976        30
weighted avg     0.6852    0.6333    0.5976        30

📊 GPT-2 Evaluation:
              precision    recall  f1-score   support

    negative     0.9091    1.0000    0.9524        10
     neutral     0.6000    0.9000    0.7200        10
    positive     1.0000    0.4000    0.5714        10

    accuracy                         0.7667        30
   macro avg     0.8364    0.7667    0.7479        30
weighted avg     0.8364    0.7667    0.7479        30

